In [1]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

pd.options.display.max_columns = 999
pd.options.display.max_rows = 1999
pd.options.display.max_colwidth = 200


In [2]:
def get_tables(tables):
    df = pd.DataFrame()
    for tb in tables:

        if type(tb.columns) == pd.core.indexes.multi.MultiIndex:
            cols = [col[0] for col in tb.columns]
            tb.columns = cols

        if "TOTALIZAÇÃO" not in tb.columns:
            tipo = tb.columns[0]
            unnamed_col = [col for col in tb if "Unnamed" in col]
            unnamed = unnamed_col[0] if unnamed_col != [] else ""

            if unnamed != "":
                tb = tb[tb[unnamed].isnull()]
                tb = tb.drop(unnamed_col, 1)
            tb["TIPO"] = tipo
            tb = tb.rename(columns={tipo: "ESCOLAS"})
            df = pd.concat([df, tb])

    cols = ["ESCOLAS", "JURADO 01", "JURADO 02", "JURADO 03", "JURADO 04", "TIPO"]
    for col in cols:
        if col not in df.columns:
            df[col] = np.nan

    return df[cols]


def pos_treatment(df):

    cols = ["escola", "jurado_1", "jurado_2", "jurado_3", "jurado_4", "tipo", "ano"]
    df.columns = cols

    cols = ["ano", "escola", "tipo", "jurado_1", "jurado_2", "jurado_3", "jurado_4"]
    df = df[cols]

    mask = df["jurado_1"].notnull()
    df = df[mask]

    mask = df["jurado_1"].str.contains("Jurado")
    df = df[np.logical_not(mask)]

    cols = ["jurado_1", "jurado_2", "jurado_3", "jurado_4"]
    for col in cols:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    return df


In [3]:
anos = list(range(1968, 2021)) + list(range(2022,2023))
df_final = pd.DataFrame()

for ano in anos:
    url = f"https://galeriadosamba.com.br/carnaval/{ano}/notas/"
    print(ano, url)
    try:
        tables = pd.read_html(url)
        dd = get_tables(tables)
        dd["ano"] = ano
        df_final = pd.concat([df_final, dd])
    except Exception:
        pass

df = pos_treatment(df_final)

1968 https://galeriadosamba.com.br/carnaval/1968/notas/
1969 https://galeriadosamba.com.br/carnaval/1969/notas/
1970 https://galeriadosamba.com.br/carnaval/1970/notas/
1971 https://galeriadosamba.com.br/carnaval/1971/notas/
1972 https://galeriadosamba.com.br/carnaval/1972/notas/
1973 https://galeriadosamba.com.br/carnaval/1973/notas/
1974 https://galeriadosamba.com.br/carnaval/1974/notas/
1975 https://galeriadosamba.com.br/carnaval/1975/notas/
1976 https://galeriadosamba.com.br/carnaval/1976/notas/
1977 https://galeriadosamba.com.br/carnaval/1977/notas/
1978 https://galeriadosamba.com.br/carnaval/1978/notas/
1979 https://galeriadosamba.com.br/carnaval/1979/notas/
1980 https://galeriadosamba.com.br/carnaval/1980/notas/
1981 https://galeriadosamba.com.br/carnaval/1981/notas/
1982 https://galeriadosamba.com.br/carnaval/1982/notas/
1983 https://galeriadosamba.com.br/carnaval/1983/notas/
1984 https://galeriadosamba.com.br/carnaval/1984/notas/
1985 https://galeriadosamba.com.br/carnaval/1985

In [4]:
# df.to_csv('historico_notas_carnaval_rio.csv', index=False)
df = pd.read_csv('historico_notas_carnaval_rio.csv')

In [5]:
df.head()

,ano,escola,tipo,jurado_1,jurado_2,jurado_3,jurado_4
0,1968,Independentes do Leblon,ALEGORIAS E ADEREÇOS,5.0,NaN,NaN,NaN
1,1968,Estácio de Sá,ALEGORIAS E ADEREÇOS,6.0,NaN,NaN,NaN
2,1968,Unidos de Lucas,ALEGORIAS E ADEREÇOS,7.0,NaN,NaN,NaN
3,1968,Unidos de Vila Isabel,ALEGORIAS E ADEREÇOS,5.0,NaN,NaN,NaN
4,1968,Portela,ALEGORIAS E ADEREÇOS,10.0,NaN,NaN,NaN
